In [11]:
import os
from glob import glob
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from dotenv import load_dotenv
load_dotenv()

import textwrap
from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Initialize variables
documents = []
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Define the directory containing the PDF files
pdf_directory = './data'

In [12]:
# pdf를 사용해서 pdf(논문)을 모두 로드
pdf_files = glob(os.path.join(pdf_directory, '*.pdf'))

# Load all PDF files using PyPDFLoader
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    pdf_documents = loader.load()
    documents.extend(pdf_documents)
    
# 텍스트는 RecursiveCharacterTextSplitter를 사용하여 분할
chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = chunk_splitter.split_documents(documents)

# embeddings은 OpenAI의 임베딩을 사용
# vectordb는 chromadb사용함

embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
vectordb = Chroma.from_documents(documents=chunks, embedding=embeddings)
retriever = vectordb.as_retriever()

In [16]:
import json

class User:
    def __init__(self, info_dir):
        # Read the JSON file
        with open(info_dir, 'r') as file:
            data = json.load(file)
        self.name = data['personal_information']['name']
        self.phone = data['personal_information']['phone']
        self.age = data['personal_information']['age']
        self.gender = data['personal_information']['gender']
        self.merry = data['personal_information']['merry']
        self.children = data['personal_information']['children']
        self.religion = data['personal_information']['religion']
        self.economy_states = data['personal_information']['economy_states']
        self.health_states = data['personal_information']['health_states']
        self.living_arrangement = data['personal_information']['living_arrangement']
import re

# Specify the path to the JSON file
info_dir = 'data/ipact_personal_main/data3.json'
# 파일 이름 문자열만 추출
file_name = info_dir.split('/')[-1]

# 정규 표현식을 사용하여 숫자 추출
data_num = int(re.findall(r'\d+', file_name)[0])

user = User(info_dir)
print(user.religion)

0


In [14]:
import json
import os

class KGLSData:
    def __init__(self, file_path):
        # 각 파일의 데이터를 초기화합니다.
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        self.kgls = data['object_data']
        self.personal_info = data['personal_information']
        self.subjective_questions = data['subjective_questions']
        self.set_reversed_questions()
        self.QA_form=self.get_user_QA_form()

    #q_num에 해당하는 문항
    def get_question(self, q_num):
        for item in self.kgls:
            if item['q_num'] == q_num:
                return item['question']
        return None
    #q_num에 해당하는 점수
    def get_user_choice(self, q_num):
        for item in self.kgls:
            if item['q_num'] == q_num:
                return item['user_choose']
        return None
    #역문항 처리함수
    def set_reversed_questions(self):
        #역문항 q_num : 1, 2, 4, 6, 7, 9, 10, 11, 13, 14
        reversed_questions=[1, 2, 4, 6, 7, 9, 10, 11, 13, 14]
        for item in self.kgls:
            if item['q_num'] in reversed_questions:
                item['user_choose']=5-int(item['user_choose'])


    def get_all_questions(self):
        return [item['question'] for item in self.kgls]
    
    def get_all_user_choices(self):
        return [item['user_choose'] for item in self.kgls]
    
    def get_all_user_choices_sum(self):
        return sum(item['user_choose'] for item in self.kgls)
    
    def get_family_user_choices_sum(self):
        # 가족관계에 관련된 문항의 q_num : 1, 6, 8, 10, 14
        family_q_nums = [1, 6, 8, 10, 14]
        return sum(item['user_choose'] for item in self.kgls if item['q_num'] in family_q_nums)
    
    def get_social_user_choices_sum(self):
        # 사회적 관계에 관련된 문항의 q_num : 2, 4, 5, 7, 9, 11
        social_q_nums = [2, 4, 5, 7, 9, 11]
        return sum(item['user_choose'] for item in self.kgls if item['q_num'] in social_q_nums)
    
    def get_belonging_user_choices_sum(self):
        # 소속감 관계에 관련된 문항의 q_num : 3, 12, 13
        belonging_q_nums = [3, 12, 13]
        return sum(item['user_choose'] for item in self.kgls if item['q_num'] in belonging_q_nums)
    
    def get_negative_user_choices_sum(self):
        negative_q_nums=[1,2,4,6, 7,9, 10, 11, 13, 14]
        return sum(item['user_choose'] for item in self.kgls if item['q_num'] in negative_q_nums)

    def get_user_QA_form(self):
        QA_form=''
        for item in self.kgls:
            QA_form+=str(item['q_num'])+" "+item['question']+": 이 대화의 점수는"+str(item['user_choose'])+ "이며 답변의 결과는 논문을 참고했을 때 낮음/중간/높음 정도이며 ...하고 ..한 결과이다. ...\\ \n"
        return QA_form


# 파일 경로 지정
directory_path = './data/ipact_personal_main'
file_paths = [os.path.join(directory_path, f'data{i}.json') for i in range(1, 11)]

# 각 파일의 데이터를 개별적으로 KGLSData 인스턴스로 생성
kgls_datasets = [KGLSData(file_path) for file_path in file_paths]

# 예시 출력 (첫 번째 데이터셋)
kgls_data = kgls_datasets[data_num-1]

print(kgls_data.get_question(1))       # "나는 가족들과 매일 대화를 나눈다."
print(kgls_data.get_user_choice(1))    # 사용자 선택에 해당하는 값
print(kgls_data.get_all_questions())   # 모든 질문 리스트 출력
print(kgls_data.get_all_user_choices_sum())  # 모든 사용자의 선택 총합
print(kgls_data.get_family_user_choices_sum())  # 가족관계 관련 문항의 선택 총합
print(kgls_data.get_social_user_choices_sum())  # 사회적 관계 관련 문항의 선택 총합
print(kgls_data.get_belonging_user_choices_sum())  # 사회적 관계 관련 문항의 선택 총합
print(kgls_data.get_negative_user_choices_sum())
# 다른 파일의 데이터셋을 사용하려면 kgls_datasets[index]로 접근하면 됩니다.


TypeError: unsupported operand type(s) for -: 'list' and 'int'

In [ ]:
KGLS="""
##KGLS 설명
본 모델의 의도에 맞게 출력을 위한 기준을 작성한 설명서입니다.
[외로움 구분 점수 기준]
*본 연구에서 정한 기준입니다.
*수치상의 비율로 구분한 점수이니 참고 바랍니다.
  14점 ~ 24점: 낮은 외로움
  25점 ~ 35점: 중간 정도의 외로움 
  36점 ~ 46점: 높은 외로움
  47점 ~ 56점: 매우 높은 외로움

[KGLS 개별 문항 점수기준]
*수치상의 비율로 구분한 점수이니 참고 바랍니다.

#본 연구에서 개발된 한국 노인의 외로움 측정도구 점수

# 가족관계 외로움 점수
- 가족관계 외로움 5문항(1번, 6번, 8번, 10번, 14번)이며 가족관계 외로움의 총점은 5점~20점 사이입니다.
5~10점은 낮은, 11~15점은 중간, 16~20점은 높은 외로움으로 말해도 좋습니다.

# 사회적 외로움 점수
- 사회적 외로움 6문항(2번, 4번, 5번, 7번, 9번, 11번)이며 가족관계 외로움의 총점은 6점~24점 사이입니다.
6~12점은 낮은, 13~18점은 중간, 19~24점은 높은 외로움으로 말해도 좋습니다.

# 소속감 외로움 점수
- 소속감 3문항은(3번, 12번, 13번)이며 소속감에 대한 외로움의 총점은 3점~12점 사이입니다.
3~5점은 낮은, 6~9점은 중간, 10~12점은 높은 외로움으로 봐도 괜찮습니다.

# 역문항에 관하여
- 역문항은 특정 함수를 통해서 역코딩을 합니다. 아래와 같은 방법을 따릅니다.
역문항 역코딩 변환식 :5-(역문항 점수)=외로움 점수
*자체적으로 세운 역문항 처리 기준입니다.

*비율은 낮음/중간/높음에 대하여 동일하게 나누었습니다.
[외로움의 정도에 대하여]
*수치상의 비율로 구분한 정도이니 참고 바랍니다.
외로움이 낮은 정도라는 것은 사용자가 적은 정도의 외로움을 느낀다는 것이며 안정적인 상태에 가까운 것입니다.
외로움이 중간 정도라는 것은 사용자가 중간 정도의 외로움을 느낀다는 것이며 안정적인 상태에 중도적인 것입니다.
외로움이 높은 정도라는 것은 사용자가 높은 정도의 외로움을 느낀다는 것이며 안정적인 상태에 먼 것입니다.

[점수 답변]
KGLS 14문항
구분	문항
1	나는 가족들과 매일 대화를 나눈다.	
2	나는 가깝게 지내는 사람들이 있다.
3	나는 쓸모없는 사람이라고 느껴진다.
4	나를 이해해주는 사람이 있다.	
5	사람들은 나와 겉으로만 어울리는 것 같다.
6	나는 자녀에게 고민을 이야기할 수 있다.
7	나는 주변 사랑과의 관계가 만족스럽다.
8	가족들은 나를 예전처럼 대해주지 않는다.
9	친구나 이웃들은 나에게 관심을 둔다.	
10	내가 아플 때 나를 보살펴줄 가족이 있다.	
11	나는 의지할 친구가 있다.
12	나는 온종일 할 일없이 시간을 보낸다.
13	나는 사회에서 필요한 사람이다.
14	나는 가족에게 의지할 수 있다.


"""
personal_info="""
#설문지 인적 정보 양식
본 설문지의 인적 정보의 답변 값에 대한 숫자와 값을 일치시켰습니다. 괄호 안의 값을 참고하세요
이름(name) : 00
연령(age) : 0(65-74세) / 1(75-84세) / 2(85세 이상)
연락처(phone) : 000-0000-0000
성별(gender) : male / female
결혼 상태(merry) : 0(미혼) / 1(기혼) / 2(이혼/별거) / 3(사별)
거주 형태(living_arrangement) : 0(혼자 거주) / 1(동거)
자녀 수 (chlidren) : 0(없음) / 2(1-2명) / 3(3명 이상)
종교(religion): 1(있음) / 0(없음)
지각된 경제 상태(economy_states): 0(나쁨) / 1(보통) / 2(좋음)
지각된 건강 상태(health_states): 0(나쁨) / 1(보통) / 2(좋음)
"""
#기존 승우님 코드에서 income은 삭제, 거주 형태가 추가되었습니다, 참고 바랍니다.
#출력 양식
OUTPUT_FORMAT=f"""
### KGLS 기반 답변 분석 \\
KGLS 14문항은 likert 4점 척도로 1점~4점 사이의 값으로 사용자의 답변을 받습니다.\n \\
점수와 외로움의 심한 정도는 비례합니다. \n \\
#### [점수 산출]
  - KGLS 점수: {kgls_data.get_all_user_choices_sum()}점
  - 가족관계 외로움: {kgls_data.get_family_user_choices_sum()}점
  - 사회적 외로움: {kgls_data.get_social_user_choices_sum()}점
  - 소속감에 대한 외로움: {kgls_data.get_belonging_user_choices_sum()}점

#### [분석 결과]
- 000 님의 KGLS 점수는 00점입니다.\\
- 가족관계 외로움은 ..., ...한 특징을 가지고 있으며, 사용자의 경우 ...한 경험으로 인해 나타납니다..\\
사용자의 가족관계 외로움의 총합은 {kgls_data.get_family_user_choices_sum()}점 입니다. 따라서 ... 수치가 ...기에 ...하며 ...하다고 볼 수 있습니다.  \\
- 사회적 외로움은 ..., ...한 특징을 가지고 있으며, 사용자의 경우 ...한 경험으로 인해 나타납니다..\\
사용자의 사회적 외로움의 총합은  {kgls_data.get_social_user_choices_sum()}점 입니다. 따라서 ... 수치가 ...기에 ...하며 ...하다고 볼 수 있습니다.  \\
- 소속감에 대한 외로움은 ..., ...한 특징을 가지고 있으며, 사용자의 경우 ...한 경험으로 인해 나타납니다..\\
사용자의 소속감에 대한 외로움의 총합은 {kgls_data.get_belonging_user_choices_sum()}점 입니다 따라서 ... 수치가 ...기에 ...하며 ...하다고 볼 수 있습니다.  \\

#### [판단 근거]
  - 가족관계 외로움 분석: 가족과의 상호작용 부족, 자녀와의 대화 부족, 가족의 보살핌 부족, ... 등의 경험으로 인해 가족관계 외로움이 낮음/중간/높음 정도입니다.\\
  - 사회적 외로움 분석: 사회적 관계의 부족/주변 사람들과의 관계 만족도 부족/친구나 이웃의 관심 부족 등의 경험으로 인해 사회적 외로움이 낮음/중간/높음 정도입니다.\\
  - 소속감 외로움 분석: 사회적 역할의 부족/쓸모없다고 느끼는 경험/온종일 할 일 없이 시간을 보내는 경험으로 인해 소속감에 대한 외로움이 낮음/중간/높음 정도입니다.\\

#### [KGLS 14 문항에 대한 외로움의 분석과 설명] \\
{kgls_data.QA_form}
  [판단 근거]   \\
  - ...\\
  - ...\\
  - ...\\
"""
SYS_PROMPT = f"""
  사용자의 인적정보, KGLS 질문과 한국형 외로움에 대한 연구(KGLS 논문)를 바탕으로 어떤 한국형 외로움을 가지고있는지, KGLS 점수를 통해 분석 결과를 도출하고 시스템 프롬프트 설정을 하는 시스템입니다.\\
  아래의 논문과 정보를 기준으로 답변해주세요. \\
  
  참고
  - {personal_info}는 인적정보의 value 값에 대한 변환을 정리하였습니다. 괄호안의 값을 참고하세요. \\
  - {KGLS}은 좀 더 모델에 특화된 KGLS기반 평가 기준입니다. \\
  
  원하는 답변
    사용자의 외로움
      - 논문을 바탕으로 사용자의 한국형 외로움에 대해 해석해주세요. \\
      
    사용자가 취약한 외로움
      - 사용자가 취약한 외로움에 대해서 알려주세요. 인적사항에 대해 큰 가중치를 부여해주세요. \\
  

  사용자 인적 정보\\
  - 이름: {user.name}
  - 나이 : {user.age}
  - 성별 : {user.gender}
  - 거주 형태 : {user.living_arrangement},
  - 결혼 여부 : {user.merry}
  - 자녀 수 : {user.children}
  - 종교 : {user.religion}, 
  - 경제 상태 : {user.economy_states}
  - 건강 상태 : {user.health_states} \\
  
"""

INPUT_PROMPT = f"""
라고 대답했습니다. \\
이 사용자가 가지고 있는 한국형 외로움을 설명해주세요. \\
그리고 어떤 대화 상대가 되어주어야 하는지 출력해주세요. \\
"""

In [ ]:
# 필요한 라이브러리 및 모듈을 임포트합니다.
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 프롬프트 템플릿을 정의합니다.
# SYS_PROMPT는 시스템 메시지로, 템플릿에 포함됩니다. 
# {context}와 {question}은 실행 시 동적으로 채워질 자리표시자입니다.
template = SYS_PROMPT + '''
3.외로움 파악을 위한 KGLS 객관식 14문항과 사용자가 입력한 답변={question} \\
4. 한국형 노인 외로움의 지표인 KGLS={context}\\

출력은 앞서 입력된 내용을 바탕으로 출력해야합니다. 특히 분석 결과는 3,4번 항목을 참고하여 충분히 설명해야만 합니다.
출력 예시:
'''+OUTPUT_FORMAT
     

# ChatPromptTemplate.from_template() 메서드를 사용하여 프롬프트 템플릿을 생성합니다.
prompt = ChatPromptTemplate.from_template(template)

# ChatOpenAI 인스턴스를 생성하여 LLM (대규모 언어 모델)을 설정합니다.
# 여기서는 'gpt-4o' 모델을 사용하고, temperature는 0으로 설정하여 출력의 일관성을 높입니다.
model = ChatOpenAI(model='gpt-4o', temperature=0)

# 문서들을 형식화하는 함수를 정의합니다.
# 각 문서의 페이지 내용을 합쳐 하나의 문자열로 반환합니다.
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

# RAG (Retrieval-Augmented Generation) 체인을 연결합니다.
# 이 체인은 문서 검색, 형식화, 프롬프트 적용, 모델 호출, 출력 파싱의 과정을 거칩니다.
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}  # 'context'는 retriever와 format_docs를 통해 설정되고, 'question'은 그대로 전달됩니다.
    | prompt  # 프롬프트 템플릿을 적용합니다.
    | model  # 모델을 호출합니다.
    | StrOutputParser()  # 출력 파서를 통해 모델의 출력을 문자열로 변환합니다.
)

# 체인을 실행합니다.
# 입력 메시지는 질문과 답변 형식의 텍스트입니다.
input_message =  f"""
Q: KGLS 답변 결과와 인적사항을 입력해주세요! \\
A: 네 알겠습니다! 양식은 다음과 같습니다
    KGLS data format\\
    {kgls_data.get_all_questions()} 이게 KGLS질문이에요.\\
    {kgls_data.get_all_user_choices()} 이게 KGLS질문에 대한 사용자의 답변이에요.\\\\
""" + INPUT_PROMPT  # 추가적인 입력 프롬프트가 이어집니다.

# to_markdown() 함수를 호출하여 체인의 결과를 마크다운 형식으로 변환합니다.
output=rag_chain.invoke(input_message)
to_markdown(output)


> ### KGLS 기반 답변 분석
> 
> KGLS 14문항은 Likert 4점 척도로 1점~4점 사이의 값으로 사용자의 답변을 받습니다. 점수와 외로움의 심한 정도는 비례합니다.
> 
> #### [점수 산출]
>   - KGLS 점수: 32점
>   - 가족관계 외로움: 12점
>   - 사회적 외로움: 13점
>   - 소속감에 대한 외로움: 7점
> 
> #### [분석 결과]
> - 이경자 님의 KGLS 점수는 32점입니다.
> - 가족관계 외로움은 가족과의 상호작용 부족, 자녀와의 대화 부족, 가족의 보살핌 부족 등의 경험으로 인해 나타납니다. 사용자의 가족관계 외로움의 총합은 12점입니다. 따라서 중간 정도의 외로움을 느끼고 있으며, 이는 가족과의 관계에서 어느 정도의 불안정성을 나타냅니다.
> - 사회적 외로움은 사회적 관계의 부족, 주변 사람들과의 관계 만족도 부족, 친구나 이웃의 관심 부족 등의 경험으로 인해 나타납니다. 사용자의 사회적 외로움의 총합은 13점입니다. 따라서 중간 정도의 외로움을 느끼고 있으며, 이는 사회적 관계에서 어느 정도의 불안정성을 나타냅니다.
> - 소속감에 대한 외로움은 사회적 역할의 부족, 쓸모없다고 느끼는 경험, 온종일 할 일 없이 시간을 보내는 경험으로 인해 나타납니다. 사용자의 소속감에 대한 외로움의 총합은 7점입니다. 따라서 중간 정도의 외로움을 느끼고 있으며, 이는 소속감에서 어느 정도의 불안정성을 나타냅니다.
> 
> #### [판단 근거]
>   - 가족관계 외로움 분석: 가족과의 상호작용 부족, 자녀와의 대화 부족, 가족의 보살핌 부족 등의 경험으로 인해 가족관계 외로움이 중간 정도입니다.
>   - 사회적 외로움 분석: 사회적 관계의 부족, 주변 사람들과의 관계 만족도 부족, 친구나 이웃의 관심 부족 등의 경험으로 인해 사회적 외로움이 중간 정도입니다.
>   - 소속감 외로움 분석: 사회적 역할의 부족, 쓸모없다고 느끼는 경험, 온종일 할 일 없이 시간을 보내는 경험으로 인해 소속감에 대한 외로움이 중간 정도입니다.
> 
> #### [KGLS 14 문항에 대한 외로움의 분석과 설명]
> 1. 나는 가족들과 매일 대화를 나눈다.: 이 문항의 점수는 2이며, 답변의 결과는 중간 정도의 외로움을 나타냅니다.
> 2. 나는 가깝게 지내는 사람들이 있다.: 이 문항의 점수는 2이며, 답변의 결과는 중간 정도의 외로움을 나타냅니다.
> 3. 나는 쓸모없는 사람이라고 느껴진다.: 이 문항의 점수는 2이며, 답변의 결과는 중간 정도의 외로움을 나타냅니다.
> 4. 나를 이해해주는 사람이 있다.: 이 문항의 점수는 1이며, 답변의 결과는 낮은 정도의 외로움을 나타냅니다.
> 5. 사람들은 나와 겉으로만 어울리는 것 같다.: 이 문항의 점수는 1이며, 답변의 결과는 낮은 정도의 외로움을 나타냅니다.
> 6. 나는 자녀에게 고민을 이야기할 수 있다.: 이 문항의 점수는 1이며, 답변의 결과는 낮은 정도의 외로움을 나타냅니다.
> 7. 나는 주변 사람과의 관계가 만족스럽다.: 이 문항의 점수는 1이며, 답변의 결과는 낮은 정도의 외로움을 나타냅니다.
> 8. 가족들은 나를 예전처럼 대해주지 않는다.: 이 문항의 점수는 4이며, 답변의 결과는 높은 정도의 외로움을 나타냅니다.
> 9. 친구나 이웃들은 나에게 관심을 둔다.: 이 문항의 점수는 4이며, 답변의 결과는 높은 정도의 외로움을 나타냅니다.
> 10. 내가 아플 때 나를 보살펴줄 가족이 있다.: 이 문항의 점수는 4이며, 답변의 결과는 높은 정도의 외로움을 나타냅니다.
> 11. 나는 의지할 친구가 있다.: 이 문항의 점수는 4이며, 답변의 결과는 높은 정도의 외로움을 나타냅니다.
> 12. 나는 온종일 할 일 없이 시간을 보낸다.: 이 문항의 점수는 1이며, 답변의 결과는 낮은 정도의 외로움을 나타냅니다.
> 13. 나는 사회에서 필요한 사람이다.: 이 문항의 점수는 4이며, 답변의 결과는 높은 정도의 외로움을 나타냅니다.
> 14. 나는 가족에게 의지할 수 있다.: 이 문항의 점수는 1이며, 답변의 결과는 낮은 정도의 외로움을 나타냅니다.
> 
> #### [판단 근거]
>   - 가족관계 외로움 분석: 가족과의 상호작용 부족, 자녀와의 대화 부족, 가족의 보살핌 부족 등의 경험으로 인해 가족관계 외로움이 중간 정도입니다.
>   - 사회적 외로움 분석: 사회적 관계의 부족, 주변 사람들과의 관계 만족도 부족, 친구나 이웃의 관심 부족 등의 경험으로 인해 사회적 외로움이 중간 정도입니다.
>   - 소속감 외로움 분석: 사회적 역할의 부족, 쓸모없다고 느끼는 경험, 온종일 할 일 없이 시간을 보내는 경험으로 인해 소속감에 대한 외로움이 중간 정도입니다.
> 
> ### 대화 상대가 되어주기 위한 조언
> 이경자 님은 중간 정도의 외로움을 느끼고 있으며, 특히 가족관계와 사회적 관계에서 외로움을 느끼고 있습니다. 따라서 다음과 같은 대화 상대가 되어주는 것이 좋습니다:
> - 가족과의 대화를 자주 시도하고, 자녀와의 소통을 늘리도록 노력합니다.
> - 사회적 활동에 참여할 수 있도록 격려하고, 친구나 이웃과의 관계를 강화할 수 있는 기회를 제공합니다.
> - 이경자 님이 쓸모없다고 느끼지 않도록, 사회적 역할을 부여하고, 그분의 경험과 지혜를 존중하는 태도를 보입니다.
> - 건강 상태와 경제 상태에 대한 걱정을 덜어줄 수 있는 지원을 제공합니다.

In [ ]:
import re
# 예시 출력 양식

with open('data/kgls_output.txt', 'w') as file:
    file.write(output)

++++

In [ ]:
import json
class QA:
    def __init__(self,file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            self.data = json.load(file)
    def process(self):
        self.data = self.data['subjective_questions']
        return self.data

qa = QA('./data/ipact_personal_main/data2.json')
print(qa.process())
class KGLSData:
    def __init__(self, file_path):
        # JSON 파일 읽기
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        self.kgls = data['KGLS']
        
    def get_question(self, q_num):
        for item in self.kgls:
            if item['q_num'] == q_num:
                return item['question']
        return None
    
    def get_user_choice(self, q_num):
        for item in self.kgls:
            if item['q_num'] == q_num:
                return item['user_choose']
        return None
    
    def get_all_questions(self):
        return [item['question'] for item in self.kgls]
    def get_all_user_choices(self):
        return [item['user_choose'] for item in self.kgls]
# 파일 경로 지정
file_path = './data/kgls_dummy.json'
kgls_data = KGLSData(file_path)

# 예시 출력
print(kgls_data.get_question(1))       # "나는 가족들과 매일 대화를 나눈다."
print(kgls_data.get_user_choice(1))    # 3
print(kgls_data.get_all_questions())   # 모든 질문 리스트 출력

with open("data/prompt_output.txt", "r") as f:
    KGLS = f.read()
personal_info="""
#설문지 인적 정보 양식
본 설문지의 인적 정보의 답변 값에 대한 숫자와 값을 일치시켰습니다. 괄호 안의 값을 참고하세요
이름(name) : 00
연령(age) : 0(65-74세) / 1(75-84세) / 2(85세 이상)
성별(gender) : male / female
결혼 상태(merry) : 0(미혼) / 1(기혼) / 2(이혼/별거) / 3(사별)
거주 형태(living_arrangement) : 0(혼자 거주) / 1(동거)
자녀 수 (chlidren) : 0(없음) / 2(1-2명) / 3(3명 이상)
종교(religion): 1(있음) / 0(없음)
지각된 경제 상태(economy_states): 0(나쁨) / 1(보통) / 2(좋음)
지각된 건강 상태(health_states): 0(나쁨) / 1(보통) / 2(좋음)
"""
SYS_PROMPT = f"""
    사용자의 한국형 외로움 분석과 인공지능 대화 상대가 되기위한 시스템 프롬프트 설정을 위한 시스템입니다. 아래의 정보와 입력된 정보를 활용하세요. 반드시 아래의 정보와 문맥을 참고하세요.\\
    1. 사용자의 인적정보
    2. KGLS 질문과 사용자의 답변의 분석결과
    3. 외로움 파악을 위한 주관식 질문사용자가 입력한 답변
    4. 노인들의 외로움과 한국형 외로움에 대한 연구
    
    {personal_info}
    사용자의 인적정보 = 이름: {user.name},
        나이: {user.age}, 
        성별: {user.gender}, 
        결혼 여부: {user.merry}, 
        자녀 수: {user.children}, 
        종교: {user.religion}, 
        거주 형태: {user.living_arrangement},
        인지된 경제 상태: {user.economy_states}, 
        인지된 건강 상태: {user.health_states},
        
        \\
    KGLS 질문과 사용자의 답변의 분석결과 = {KGLS} \\

    
    
"""
INPUT_PROMPT = f"""

"""

{'loneliness_situations': '아픈데 혼자 식사를 할 때', 'realization_of_loneliness': '누워서 생각하면 이 생각 저 생각으로 외로움을 느낌', 'needs_during_loneliness': '주변인들과 왕래가 잦아야하고,회관에서 하는 프로그램이 많았으면 함'}
나는 가족들과 매일 대화를 나눈다.
3
['나는 가족들과 매일 대화를 나눈다.', '나는 가깝게 지내는 사람들이 있다.', '나는 쓸모없는 사람이라고 느껴진다.', '나를 이해해주는 사람이 있다.', '사람들은 나와 겉으로만 어울리는 것 같다.', '나는 자녀에게 고민을 이야기할 수 있다.', '나는 주변 사랑과의 관계가 만족스럽다.', '가족들은 나를 예전처럼 대해주지 않는다.', '친구나 이웃들은 나에게 관심을 둔다.', '내가 아플 때 나를 보살펴줄 가족이 있다.', '나는 의지할 친구가 있다.', '나는 온종일 할 일없이 시간을 보낸다.', '나는 사회에서 필요한 사람이다.', '나는 가족에게 의지할 수 있다.']


In [ ]:
# 필요한 라이브러리 및 모듈을 임포트합니다.
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 프롬프트 템플릿을 정의합니다.
# SYS_PROMPT는 시스템 메시지로, 템플릿에 포함됩니다. 
# {context}와 {question}은 실행 시 동적으로 채워질 자리표시자입니다.
template = SYS_PROMPT + '''
    3. 외로움 파악을 위한 주관식 질문과 사용자가 입력한 답변 = {question} \\
    4. 노인들의 외로움과 한국형 외로움에 대한 연구 = {context}\\
    
    출력은 앞서 입력된 내용을 바탕으로 출력해야합니다. 특히 분석결과는 3,4번 항목을 참고하여 충분히 설명해야하며, 한국형 외로움의 질적연구에서 주관적 답변을 분석했던 결과를 바탕으로 사용자의 외로움을 예측하고 해당하는 외로움들을 출력해야합니다.\\
    출력 예시:
    [외로움 종류, 분석 결과]
    - 000 님이 해당하는 한국형 외로움은 OOO 00%, OOO 00% ...입니다. 
    - OOO 외로움은 ..., ...한 특징을 가지고 있으며, 사용자의 경우 ...한 경험으로 인해 나타납니다..\\
    - OOO 외로움은 ..., ...한 특징을 가지고 있으며, 사용자의 경우 ...한 경험으로 인해 나타납니다..\\
    - ...
    [판단 근거]
    - 외로움 종류에 대한 설명
    - 외로움에 관한 연구논문에서 찾은 판단 근거
    
    [대화 시스템 프롬프트]
    - 당신은 000님을 위한 대화 상대가 되어주어야합니다. 아래의 참고사항을 반영해 출력하세요.
    - 대화중 사용자의 기억은 반드시 기억해야합니다.
    - 말투는 ...하고, ...해야할 것입니다. 
    - ...
'''

# ChatPromptTemplate.from_template() 메서드를 사용하여 프롬프트 템플릿을 생성합니다.
prompt = ChatPromptTemplate.from_template(template)

# ChatOpenAI 인스턴스를 생성하여 LLM (대규모 언어 모델)을 설정합니다.
# 여기서는 'gpt-4o' 모델을 사용하고, temperature는 0으로 설정하여 출력의 일관성을 높입니다.
model = ChatOpenAI(api_key=OPENAI_API_KEY,model='gpt-4o', temperature=0)

# 문서들을 형식화하는 함수를 정의합니다.
# 각 문서의 페이지 내용을 합쳐 하나의 문자열로 반환합니다.
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

# RAG (Retrieval-Augmented Generation) 체인을 연결합니다.
# 이 체인은 문서 검색, 형식화, 프롬프트 적용, 모델 호출, 출력 파싱의 과정을 거칩니다.
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}  # 'context'는 retriever와 format_docs를 통해 설정되고, 'question'은 그대로 전달됩니다.
    | prompt  # 프롬프트 템플릿을 적용합니다.
    | model  # 모델을 호출합니다.
    | StrOutputParser()  # 출력 파서를 통해 모델의 출력을 문자열로 변환합니다.
)

# 체인을 실행합니다.
# 입력 메시지는 질문과 답변 형식의 텍스트입니다.

input_message = str(qa) + INPUT_PROMPT  # 추가적인 입력 프롬프트가 이어집니다.
text = rag_chain.invoke(input_message)
# to_markdown() 함수를 호출하여 체인의 결과를 마크다운 형식으로 변환합니다.
print(text)


[외로움 종류, 분석 결과]
- 이경자 님이 해당하는 한국형 외로움은 "중간 정도의 외로움" 75%, "외부관계에서 오는 외로움" 60%, "혼자라고 느껴지는 쓸쓸함" 50%입니다.
- "중간 정도의 외로움"은 가족과 떨어져 지내는 경험, 건강 상태의 악화, 경제적 어려움 등으로 인해 나타납니다. 이경자 님의 경우, 이혼 후 혼자 지내는 시간이 많아지면서 이러한 외로움을 느끼고 있습니다.
- "외부관계에서 오는 외로움"은 가족, 친구, 사회적 관계의 부족으로 인해 느끼는 외로움입니다. 이경자 님은 자녀들이 성인이 되어 독립한 후, 가족과의 관계에서 외로움을 느끼며, 특히 혼자 남아있을 때 외로움을 크게 느낍니다.
- "혼자라고 느껴지는 쓸쓸함"은 배우자와의 사별, 가족과의 소원한 관계 등으로 인해 느끼는 외로움입니다. 이경자 님은 이혼 후 혼자 지내는 시간이 많아지면서 이러한 쓸쓸함을 느끼고 있습니다.

[판단 근거]
- 외로움 종류에 대한 설명:
  1. "중간 정도의 외로움"은 KGLS 점수 29-42점에 해당하며, 이는 가족과의 분리, 건강 문제, 경제적 어려움 등 다양한 요인으로 인해 발생합니다.
  2. "외부관계에서 오는 외로움"은 가족, 친구, 사회적 관계의 부족으로 인해 느끼는 외로움으로, 연구에 따르면 노인들은 배우자와의 사별이나 가족과의 관계 소홀, 무관심으로 인해 외로움을 경험합니다.
  3. "혼자라고 느껴지는 쓸쓸함"은 배우자와의 사별, 가족과의 소원한 관계 등으로 인해 느끼는 외로움으로, 연구에 따르면 노인들은 배우자와의 사별로 인해 혼자 남겨진 느낌을 받으며, 견디기 힘든 외로움을 경험합니다.

- 외로움에 관한 연구논문에서 찾은 판단 근거:
  1. "중간 정도의 외로움"은 KGLS 점수 42점으로, 이는 중간 정도의 외로움에 해당합니다.
  2. "외부관계에서 오는 외로움"은 연구에서 노인들이 가족, 친구나 사회적 관계의 부족으로 인해 느끼는 외로움으로 구성되며, 이경자 님의 경우 자녀들이 독립한 후 가족과의 관계에서 외로움을 느

In [ ]:
with open('data/prompt_output.txt', 'w') as file:
    file.write(text)